<a href="https://colab.research.google.com/github/lucastanger/vehicle_classification/blob/main/vehicle_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [13]:
dfs = []

for i in range(1, 7):
  dfs.append(pd.read_csv('https://raw.githubusercontent.com/lucastanger/vehicle_classification/main/data/DE11_{}.csv'.format(i), sep=";"))

dfs

[    Takt Radar / Distanz (m) Breite (m) Reflektionsstaerke Geschwindigkeit
 0      1                61,2      0,735           0,050715          119,56
 1      2                60,8       0,63         0,03803625          119,92
 2      3                60,5       0,42           0,050715          120,64
 3      4                60,4       0,63         0,06339375          119,92
 4      5                60,1       0,63          0,0760725          119,56
 5      6                59,7      0,525          0,0760725          119,56
 6      7                59,3       0,63         0,08875125          119,56
 7      8                58,9       0,63         0,08875125          119,92
 8      9                58,6      0,735            0,10143          119,92
 9     10                58,3      0,067            0,10143          120,28
 10    11                58,1       0,63            0,10143          119,56
 11    12                57,7       0,67         0,08875125          119,92
 12    13   

In [14]:
from pyds import MassFunction

ModuleNotFoundError: ignored

In [ ]:
omega = 'plmf'